# Train CNN

In [ ]:
import tensorflow
from tensorflow.keras.layers import Conv2D, LeakyReLU, MaxPool2D, Flatten, Dropout, Dense, Input

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow.keras.models import load_model

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
# from hyperopt import space_eval

import os

import csv
import time
from tensorflow.keras.regularizers import L2

In [ ]:
savepath_model = '.'

# Functions for model definition and prediction

In [ ]:
def cnn_model(input_shape=(64, 64, 1), dropout=0.07,
              alpha=0.0, nb_blocks = 3, filter_sizes=[32, 16, 8],
              kernel_size=(3,3), nb_classes=10, l2_value=0.0):

    if not len(filter_sizes) == nb_blocks:
        raise ValueError("# filters must be compatible with nb_blocks.")

    l2_reg = L2(l2=l2_value)

    inputs = Input(shape=input_shape, name='input')

    convlayer_counter = 0
    pooling_counter = 0
    activation_counter = 0
    dropout_counter = 0
    for i, filters in enumerate(filter_sizes):
        if i == 0:
            x = inputs

        for j in range(2): # repeat the following blocks 2 times

            x = Conv2D(filters=filters, kernel_size=kernel_size, strides=(1, 1),
                       padding='same', kernel_initializer='Orthogonal',
                       data_format="channels_last", kernel_regularizer=l2_reg,
                       name='Convolution_2D_{}'.format(convlayer_counter))(x)
            convlayer_counter += 1
            x = LeakyReLU(alpha=alpha, name='Leaky_ReLU_{}'.format(activation_counter))(x)
            activation_counter += 1
            x = Dropout(rate=dropout, name='Dropout_{}'.format(dropout_counter))(x, training=True)
            dropout_counter += 1

        if not i == (nb_blocks - 1):
            # for last block, no max pooling done
            x = MaxPool2D(pool_size=(2, 2), strides=(2, 2),
                          data_format="channels_last",
                          name='MaxPooling2D_{}'.format(pooling_counter))(x)
            pooling_counter += 1

    x = Flatten(name='Flatten')(x)
    x = Dense(128, name='Dense_1', kernel_regularizer=l2_reg, activation='relu')(x)
    x = Dropout(rate=dropout,
                name='dropout_{}'.format(dropout_counter))(x, training=True)
    outputs = Dense(nb_classes, name='Dense_2', activation='softmax') (x)

    model = tensorflow.keras.Model(inputs, outputs)

    adam = Adam(beta_1=0.9, beta_2=0.999, decay=0.0)

    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['categorical_accuracy'])

    return model


In [ ]:
def decode_preds(data, model, n_iter=1000):

    results = []
    for idx in range(n_iter):
        pred = model.predict(data, batch_size=2048)
        results.append(pred)

    results = np.asarray(results)
    predictions = np.mean(results, axis=0)
    return predictions

In [ ]:
def train_and_test_model(model, X_train, y_train, X_val, y_val,
                         epochs=100, batch_size=64, verbose=1, n_iter=1000):

    callbacks_savepath = os.path.join(savepath_model, 'model_it_{}.h5'.format(ITERATION))

    callbacks = []
    monitor = 'val_loss'
    mode = 'min'
    # Alternative: monitor validation accuracy.
    #monitor = 'val_categorical_accuracy'
    #mode = 'max'    
    save_model_per_epoch = ModelCheckpoint(callbacks_savepath, monitor=monitor, verbose=1,
                                       save_best_only=True, mode=mode, period=1)
    callbacks.append(save_model_per_epoch)

    # Additional possibility: use Early stopping.
    #es = EarlyStopping(monitor=monitor, mode=mode, patience=10)
    #callbacks.append(es)

    # Fit model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        verbose=verbose, validation_data=(X_val, y_val),
                        callbacks=callbacks)

    optimal_model = load_model(callbacks_savepath)


    train_pred = decode_preds(data=X_train, model=optimal_model, n_iter=n_iter)
    acc_train = accuracy_score(y_true=y_train.argmax(axis=-1), y_pred = train_pred.argmax(axis=-1))

    val_pred = decode_preds(data=X_val, model=optimal_model, n_iter=n_iter)
    acc_val = accuracy_score(y_true=y_val.argmax(axis=-1), y_pred = val_pred.argmax(axis=-1))

    return acc_train, acc_val, optimal_model, history


# Prepare training data

In [ ]:
# Prepare training data
X_pristine = np.load(os.path.join(save_path, 'X_fft.npy'))
y_pristine = np.load(os.path.join(save_path, 'y.npy'))
                     
X_distorted = np.load(os.path.join(save_path, 'X_fft_distorted.npy'))
y_distorted = np.load(os.path.join(save_path, 'y_fft_distorted_int.npy'))
    
num_classes = np.unique(y_pristine).size

X = np.concatenate((X_pristine, X_distorted))
y = np.concatenate((y_pristine, y_distorted))

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,
                                                  random_state=42, stratify=y)


y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_val = tensorflow.keras.utils.to_categorical(y_val, num_classes=num_classes)


print(X_train.shape, X_val.shape)

# Define architecture (see scripts folder for hyperopt optimization protocol)

In [ ]:
params = {"epochs": 2, 
          "batch_size": 64,
          "alpha": 0.0, 
          "kernel_size": (3,3),
          "architecture": (3, [32, 16, 8]),
          "dropout": 0.07, 
          "l2_value": 0.0}


global ITERATION
ITERATION = 0

In [ ]:
t_start = time.time()


model = cnn_model(input_shape=(64, 64, 1), dropout=params["dropout"], alpha=params["alpha"],
                  nb_blocks=params["architecture"][0], filter_sizes=params["architecture"][1],
                  kernel_size=params["kernel_size"], nb_classes=np.unique(y_val.argmax(axis=-1)).size,
                  l2_value=params['l2_value'])

acc_training, acc_validation, model, history = train_and_test_model(model=model, batch_size=params['batch_size'],
                                                                    epochs=params['epochs'],
                                                                    X_train=X_train, y_train=y_train,
                                                                    X_val=X_val, y_val=y_val, verbose=1,
                                                                    n_iter=10)#1000)


for key in history.history:
    np.save(os.path.join(savepath_model, 'it_{}_{}.npy'.format(ITERATION, key)), history.history[key])


t_end = time.time()




eval_time = round(abs(t_start-t_end),3)


In [ ]:
print('Final validation accuracy: {}'.format(acc_validation))